### Your first hypervelocity star catalogue

`speedystar` is a python package which will allow you to generate, evolve, propagate and perform mock observations of single stars ejected at high velocities.

### Getting Started

Make sure this notebook is in the parent directory of `speedystar/`. The package itself is not large, but performing mock observations requires downloading a Milky Way dust map which by default is ~500 MB. This free space is assumed to be in the current working directory by default, but does not have to be (see below).

`speedystar` uses a number of Python packages. Some of these which might not already be included in your Python distribution are astropy, galpy, mwdust and pygaia. Run this notebook and conda or pip install any package that doesn't exist.

More accurate treatments of the Gaia selection functions and astrometric errors require Python packages which currently do not support Windows, but can be installed using Windows Subsystem for Linux (WSL). See my_gaia_selection.ipynb for more information

### Documentation

help(speedystar.starsample) will display the names and descriptions of every method in `speedystar`, as well as descriptions of common column names.

### Outputs

`speedystar` outputs are .fits tables containing useful info for each mock star as well as metadata describing the assumptions that go into the sample. They can be accessed using astropy.table or with [Topcat](https://www.star.bris.ac.uk/~mbt/topcat/) or however else you'd like to read them.


In [ ]:
#Import what you need
import numpy as np
from speedystar import starsample
from speedystar.eject import Hills, BMBH
from speedystar.utils.mwpotential import MWPotential
import astropy.units as u
from galpy import potential
from galpy.potential import MWPotential2014
import mwdust

#Print a lot of documentation
help(starsample)

A new version of galpy (1.10.2) is available, please upgrade using pip/conda/... to get the latest features and bug fixes!
Help on class starsample in module speedystar:

class starsample(builtins.object)
 |  starsample(inputdata=None, name=None, isExternal=False, **kwargs)
 |  
 |   HVS sample class. Main features:
 |  
 |   - Generate a sample of HVS with a specified ejection model
 |   - Propagate the ejection sample in the Galaxy
 |   - Obtain mock photometry for the sample in a variety of bands
 |   - Obtain mock astrometric and radial velocity errors
 |   - Subsample the population according to various criteria
 |   - Save/Load catalogues FITS files
 |  
 |  # Common attributes
 |   ---------
 |   self.size : int
 |       Size of the sample
 |   self.name : str
 |       Catalog name, 'Unknown'  by default
 |   self.ejmodel_name : str
 |       String of the ejection model used to generate the sample.
 |      'Unknown' by default
 |   self.dt : Quantity
 |       Timestep used for o

### Ejecting a sample

We're now ready to eject some stars!

The 'typical' way to eject hypervelocity stars is via the Hills Mechanism, in which a stellar binary approaches Sgr A* on a very low-angular momentum orbit. At a particular distance from Sgr A*, the tidal forces across the binary surpass the gravitational binding energy of the binary itself and the binary is separated. One star is captured in orbit around Sgr A* and the other is flung away at a velocities up to several thousand km/s. Stars ejected above the escape velocity of the Galaxy are hypervelocity stars (HVSs) -- they will eventually escape the Galaxy entirely. Note that there will also be a population of stars ejected slowly as well, which will not escape the inner Galaxy and might survive to interact again with Sgr A*.

With `speedystar` you first generate a sample of stars at the moment of ejection. The number of stars and their masses/metallicities/velocities/flight times/evolutionary stages depend to varying degrees on assumptions you make about stars and binaries in the Galactic Centre, see [Evans+2022](https://ui.adsabs.harvard.edu/abs/2022MNRAS.512.2350E/abstract) for more details.


In [2]:
#Current ejection methods include 'Hills' implementing the Hills mechanism
# and 'BMBH' implementing the massive black hole binary slingshot mechanism

# Arguments to Hills() allow you to set many parameters for the sample, e.g. the stellar initial mass function, the mass of Sgr A*, the maximum flight time, etc. See documentation for details.
ejectionmodel = Hills(rate=1e-4/u.yr)

# Eject a sample of stars from Sgr A*. 
mysample = starsample(ejectionmodel, name='My Hills catalogue')

# Save ejection sample
mysample.save('./cat_ejection.fits')

#Uncomment these lines to experiment with the BMBH
#Arguments to BMBH() allow you to set other parameters, most importantly the component masses of the binary, its separation or the time since merger. See documentation for details.
#ejectionmodel = BMBH(m_bh=4e6*u.Msun, m_c=4e3*u.Msun, current_a=0.001*u.pc)
#mysample = starsample(ejectionmodel, name='My BMBH catalogue')

# Save ejection sample
#mysample.save('./cat_ejection_BMBH.fits')

Evolving HVSs: 100%|██████████| 891/891 [00:02<00:00, 411.84it/s]


### Accessing the data 

Kinemetic/stellar (and eventually astrometric and photometric) properties are all stored as attributes and are easily accessible and modifiable. Each has assigned astropy units when relevant.

The saved .fits files are formatted as simple tables (+ metadata) so it is straightforward to load them as astropy tables or open up in other programs like Topcat if desired. 

Call `starsample.attrsummary()` to print a list of all attributes of the sample and their descriptions. These descriptions are also included in the .fits files as the column descriptions.

In [ ]:
#Set a new attribute as double the initial velocity
mysample.double_v = 2. * mysample.v0

#Set a new metavarbiable
mysample.metavar = 'foo'

#Print a summary of the attributes of the sample. User-defined attributes are included but a description will not be available.
mysample.attrsummary()

#Alternatively, call mysample.whatis('an_attribute') to get a description of a specific attribute. This will also not work for user-defined attributes.
mysample.whatis('v0')

#Note that trying to select a property as if mysample were an astropy table or pandas dataframe (e.g. mysample['v0']) will not work since this syntax is reserved for subsetting (see below). If columns need to be selected or changed programmatically, use getattr(mysample, 'an_attribute') or setattr(mysample, 'an_attribute') instead.

### Propagating the sample

The 'ejection sample' consists of a population of stars freshly ejected from the centre of the Milky Way. Next we have to propagate them through the Galaxy (each at its own assigned flight time) under an assumed potential to find out where they will be at the present day.
The final positions and velocities in Galactocentric Cartesian, Galactocentric cylindrical, Galactic, and equatorial coordinates will be stored as attributes.
Note that the entire orbit for each star is also available as an attribute, but this cannot be saved to file and will not be available if the sample is loaded from file. 

<p align="center">
<img src ="https://cdn.sci.news/images/enlarge4/image_5003e-Hypervelocity-Stars.jpg" width="25%" height="25%">
</p>


In [ ]:
# Load ejection sample, if it doesn't already exist
mysample = starsample('./cat_ejection.fits')

# Assume a Galactic potential
default_potential = MWPotential2014

#Ensure the potential has physical units so that the final positions and velocities have physical units too. This may or may not be necessary depending on the potential you are using.
potential.turn_physical_on(default_potential)

#Propagate sample. Positions / velocities are returned in Cartesian (x/y/z/v_x/v_y/v_z) and Galactic (l/b/dist/pm_l/pm_b/v_radial) and equatorial (ra/dec/dist/pm_ra/pm_dec/v_radial) coordinates
mysample.propagate(potential = default_potential)

#Save propagated sample
mysample.save('./cat_propagated.fits')

mysample.attrsummary()

#Print the present-day Galactocentric distances of the first 5 stars
print(mysample.GCdist[0:5])

#The Galactocentric x position of the first star throughout its flight time
#NOTE the time array is negative, since t=0 is now and the stars were ejected tflight in the past.
ts = np.linspace(-mysample.tflight[0], 0*u.Myr, 10)
print(mysample.orbits[0].x(ts, quantity=True))

### Subsetting the data

So far our sample includes every star. There are many ways of making selection cuts on the sample, e.g. to get only the stars unbound to the Galaxy, or only stars above a certain mass, or only the first N stars, etc.

In [ ]:
mysample = starsample('./cat_propagated.fits')
print(mysample.size)

#Call mysample.subsample() with an array of indices to create a subsample
#e.g. to only get the unbound stars
inds = np.where(mysample.GCv > mysample.Vesc)[0]
mysample.subsample(inds)

print(mysample.size)

#e.g. to get only the 5th to 9th star in the sample
mysample = starsample('./cat_propagated.fits')
inds = np.arange(5,10)
mysample.subsample(inds)

print(mysample.size)

#e.g. to get 10 stars at random
mysample = starsample('./cat_propagated.fits')
mysample.subsample(10)

print(mysample.size)

#Note that calling mysample.subsampple() makes an in-place change to mysample, i.e. any stars that do not satisfy the logical condition are removed from the object and must be reload from the original file if needed.

#To create a new subsample while leaving the original sample unchanged, mysample supports indexing.
mysample = starsample('./cat_propagated.fits')
inds = np.where(mysample.GCv > mysample.Vesc)[0]
mysubsample = mysample[inds]

print(mysample.size)
print(mysubsample.size)


### Mock observations of the sample

We next have to figure out how bright each of the stars is, otherwise we don't know which of the stars would be detectable today or in the near future. `speedystar` is able to calculate the apparent magnitudes of the sample in a variety of photometric bassbands (e.g. Gaia G/G_BP/G_RP/G_RVS, Johnson-Cousins UBVRI, SDSS/LSST ugriz, VISTA JHK) depending on the mass, temperature and radius of the stars along with their distance and position on the sky.



In [ ]:
#Load a pre-existing propagated sample, if needed
mysample = starsample('./cat_propagated.fits')

#Magnitudes are exctincted by Milky Way dust along the line of sight. Need to assign a dust map. 
#Dust map(s) must be downloaded if they do not exist. They can be rather large, ~500 MB.

#Uncomment this line and fill out a path if the dust map is located somewhere other than the working directory, or you want it downloaded somewhere other than the working directory
#mysample.config_dust('/path/to/where/you/want/the/dust/map')

#Assign the dust map. Will be downloaded if it doesn't already exist in the working directory or where you've specified above
mysample.dust = mwdust.Combined15()

#Get mock apparent magnitudes . By default magnitudes are computed in the Johnson-Cousins V and I bands and the Gaia G, G_RP, G_BP and G_RVS bands.
mysample.photometry()

#Check the attributes
mysample.attrsummary()

mysample.save('./cat_photometry.fits')

### Gaia detectability
 Finally, let's determine which stars would be detectable in Gaia Data Release 3.
 
 Gaia magnitude cuts could be performed manually, but they're also implemented directly in `speedystar.subsample` for simplicity, along with some other common cuts. This cut will also automatically calculate DR3 mock Gaia errors.

 Note that by default the Gaia subsampling is based on simplistic magnitude and colour cuts. More sophisticated cuts based on the Gaia selection functions are implemented but require some time to load in. The same can be said about the error calculation.
 See my_gaia_selection.ipynb for a walkthrough of implementing more realistic (but slightly more labourious) treatments for selecting Gaia stars and calculating their errors.


In [ ]:
#Load a pre-existing sample with photometry, if needed
mysample = starsample('./cat_photometry.fits')

#Determine which stars would be in principle detectable in Gaia DR3. Astrometric and radial velocity errors are calculated automatically.
mysample.subsample('Gaia_DR3')

#Check the attributes
mysample.attrsummary()

#Call mysample.list_cuts() to see the built-in cuts available.
mysample.list_cuts()

#Save the cut sample
mysample.save('./cat_gaiaDR3.fits')
print('Number of stars in Gaia DR3: '+str(mysample.size))

mysample = starsample('./cat_photometry.fits')

#Determine which stars would be in principle detectable in the subsample of Gaia DR4 with measured radial velocities.
mysample.subsample('Gaia_6D_DR4')

#Save the cut sample
mysample.save('./cat_gaiaDR4_6D.fits')
print('Number of stars in Gaia DR4 with radial velocity: '+str(mysample.size))

#Load a pre-existing sample with photometry, if needed
mysample = starsample('./cat_photometry.fits')

#These built-in cuts can also be called with indexing.
mysubsample = mysample['Gaia_DR3']
print('Number of stars in Gaia DR3: '+str(mysubsample.size))
